# About this code
This is an enumerator for the connected partitions of grid graphs. We might ask how many ways there are to cut an $m\times n$ grid into $k$ connected pieces, where we can consider both *rook* connectivity, meaning that two squares are considered adjacent if and only if they meet at an edge, and *queen* connectivity where we also consider two squares which meet at a corner to be adjacent.

There are a very large number of such partitions.  For the $7\times 7$ grid, there are almost 150 million ways to partition it into seven rook-connected pieces of size seven.  However, for smaller instances of the problem, we can count the number of such partitions relatively quickly, and that is what this code does.  We include the option to actually output the list of partitions as well.

This notebook walks through the algorithm and its proof-of-correctness.  The docstrings in the methods give a sketch of the functionality, and more details can be found in the written text. 

Users who want to run this code on graphs larger than the 6 by 6 grid should download it and run it from a Julia terminal.  This can be done in Jupyter by going to `File > Download As > Julia (.jl)`.  In the Jupyter nbviewer, click the "View as Code" icon in the top right (it looks like "\</\>") and save this as a `.jl` file.

### The Julia Language
This code is written in Julia, a relatively new language designed by some cool folks at MIT for scientific and numerical applications.  Julia blends the usability of Python with the (almost) speed of C with a MATLAB-like syntax.  It is just-in-time compiled, so it's not unusual for the first execution of a method (what functions and stuff are called in Julia) to be slow, since the compiler needs to do its thing, but subsequent calls are fast.  As a relative benchmark, this exact same algorithm in Python runs about ten times slower.

Julia is functional rather than object-oriented like Python, and it's typing, polymorphic methods, and homoiconism are really cool and let you do some neat things, and I definitely recommend checking out more of the language's features if you're interested.

Our implementation of this algorithm in Python runs about 10 times slower than this code.  [Bob Harris' C implementation](http://www.bumblebeagle.org/polyominoes/tilingcounting/) runs at a similar speed to this Julia code for the smaller examples.  For larger examples, Harris' code is faster and more memory-efficient, but it only counts partitions; it cannot output the enumeration.

In [ ]:
# These are the packages we need to use.  If you don't
# have one, uncomment and run in the next block the
# appropriate lines and then run this block again

using Pkg
using LightGraphs, MetaGraphs
using IterTools
using DataStructures
using Base
using Combinatorics

In [ ]:
Pkg.add("LightGraphs")
Pkg.add("IterTools")
Pkg.add("DataStructures")
Pkg.add("Combinatorics")
Pkg.add("MetaGraphs")
Pkg.add("StatsBase")

# Now to the Algorithm!

### Ominos
The first fundamental observation to make is that any valid partitioning of the grid is uniquely specified by the collection of pieces (which we call *ominos*) which compose it.  Therefore, instead of starting with the whole grid and trying to cut it up, we'll instead start with all of the pieces and figure out which are the valid ways to put those pieces together.

Given a set of ominos, a *valid $k$-partition of the $m\times n$ grid* is a collection of $k$ ominos from that set such that none overlap and they together exactly cover the grid.

We begin with several methods to construct these sets of ominos.

In [ ]:
"""
Checks if an omino is 'valid'. 
A valid omino is a binary matrix where
all of the ones are positioned such that
they form a single (rook/queen)-connected
component.
"""
function valid_omino(omino,cont="rc")
    if 2 in omino
        return false
    end
    n1 = size(omino)[1]
    n2 = size(omino)[2]
    check = zeros(Int8, n1,n2)
    _f = false
    g = Grid([n1,n2])
 
    if cont == "qc" 
        for i=1:n1-1
           for j=1:n2-1
               add_edge!(g,((i-1)*n2+j),i*n2+j+1) 
            end
        end
        
        for i=1:n1-1
            for j=2:n2
                add_edge!(g,((i-1)*n2+j),i*n2+j-1)
            end
        end
    end
    
    for i=n1:-1:1
        for j=n2:-1:1
           if omino[i,j] ==0
                rem_vertex!(g,((i-1)*n2+j))
            end  
        end   
    end
    
    return length(connected_components(g)) == 1  
end
    



"""
Given a list of indices [(i_1,j_1),(i_2,j_2),...]
checks if setting the values at those indices in
a grid of size grid[1]xgrid[2] yields a valid
omino as according to the specified continuity
rule.  

If so, returns that omino.  If not,
returns the grid[1]xgrid[2] zeros matrix.
"""
function make_omino(inds,grid,cont="rc")
    
    p = zeros(Int8,grid[1],grid[2])
    for loc in inds
        p[loc[1],loc[2]]=1
    end
    
    if valid_omino(p,cont)
        return p
    else
        return zeros(Int8,grid[1],grid[2])
    end
end


"""
Returns the set of valid grid[1]xgrid[2] ominos, 
according to the continuity rule, which have 
cells ones.

Note: starting with make_omino_set(1,[...]) and
calling biggen(...) in a loop is generally much
faster than this method.  Since this operates
by combinatorial enumeration, checking all
possible subsets of indices.
"""
function make_omino_set(cells,grid,cont="rc")
    
   pair_idx = []
    for i=1:grid[1]
        for j=1:grid[2]
            push!(pair_idx, (i,j))
        end
    end
    
    ominos = []
    for t in subsets(pair_idx, Val{cells}())
        p = make_omino(t,grid,cont)
        
        if sum(p)!=0
            push!(ominos,p)
        end
    end   
    return ominos    
end

### Building omino sets
The function `make_omino_set()` is correct, as it checks all possible binary matrices with `cells` ones for validity. However, this method is also somewhat obviously slow, since there are a very large number of binary matrices and relatively few of them are valid ominos.  By making the observation that any $(k+1)$-omino can be thought of by adding one cell to a $k$-omino, we can dramatically decrease the time needed to construct the omino sets.  This is what the following method, `biggen()` does -- it takes a set of ominos and returns the set of ominos reachable by augmenting something in that set.

If we start with the set of $1$-ominos on a grid (the set of $m\times n$ binary matrices with a single one and call `biggen()` recursively $k-1$ times, we can construct every $k$-omino.  The proof is by induction, and follows immediately from the above observation.  Since every $(k+1)$-omino is an augmented $k$-omino, if we know we have the complete set of $k$-ominos, then `biggen()` will yield the complete set of $(k+1)$-ominos.  Since we start with the complete set of $1$-ominos by assertion, the base case holds as well.



In [ ]:
"""
Takes as input a collection of ominos
and returns the list of valid ominos on
the same shape grid which can be formed
by changing a single zero to a one in
any element of omlist.
"""
function biggen(omlist,cont="rc")
    grid = size(omlist[1])
    new_oms = Set()
    for o in omlist
        
        for i = 1:grid[1]
            for j = 1:grid[2]
                z = zeros(Int8,grid[1],grid[2])
                z[i,j]+=1
                if o[i,j]==0 && valid_omino(z+o,cont)
                    push!(new_oms,z+o)                 
                end
            end
        end       
    end
    return collect(new_oms)
end

# Checking for bad holes

Since we're trying to enumerate a collection of exponentially many things, we need to use all of the tricks we can find to pare down the space of objects we have to evaluate.  One such trick is to discard ominos that are incompatible with *any* partitioning, since they have 'holes' which cannot be filled by any other omino(s).  To illustrate, consider the omino below:

 

    0 1 0
    1 1 0
    0 0 0
 

If we're looking at partitions of the $3\times 3$ grid into three pieces of size three, then this omino can't be part of any valid partition, since there's no other omino compatible with this which will let us fill in the top left corner.  As such, we should throw this one away.

This observation yields the following fact:

An omino is incompatible with any partition into pieces of size $p_1,p_2,\dots,p_\ell$ if it has a hole whose size cannot be written as the sum of $p_i$ in any way.

We'll make the additional observation that the set of sizes of holes which cannot be filled at least includes everything smaller than the minimum $p_i$ as well as anything in between the maximum $p_1$ and twice the minimum $p_i$.  We note as well that this set may be empty, as in the case where we allow the use of $1$-ominos.

In [ ]:
"""
Given an omino, and a collection of bad
hole sizes (bad_holes), returns true
if none of the connected components formed
by the ominos zeros are of a size in that
list.

For example if we're looking at splitting the 3x3
grid into 3 rook connected pieces of size 3, then we
might have bad_holes=[1,2,4,5], since it's going to
be impossible to add any collection of our 3-ominos
to this omino to complete the partitioning.  Therefore,
if we call this method on

0 1 0
1 1 0
0 0 0

it would return false.
"""
function check_holes(omino, bad_holes,cont="rc")

    n1 = size(omino)[1]
    n2 = size(omino)[2]
    check = ones(Int8, n1,n2)-omino
    _f = false
    g = Grid([n1,n2])
     
    if cont == "qc"
        for i=1:n1-1
           for j=1:n2-1
               add_edge!(g,((i-1)*n2+j),i*n2+j+1  ) 
            end
        end
        
        for i=1:n1-1
            for j=2:n2
                add_edge!(g,((i-1)*n2+j),i*n2+j-1)
            end
        end        
    end
    
    for i=n1:-1:1
        for j=n2:-1:1
           if check[i,j] ==0
                rem_vertex!(g,((i-1)*n2+j))
            end            
        end   
    end
    
    for c in connected_components(g)
        if length(c) in bad_holes
            return false
        end
    end
    
    return true
end


### Defining the tree structure

We now make another observation: given any partitioning into ominos, we can canonically order those ominos by reading them off of the grid left-to-right, top-to-bottom.  Each partition has a unique canonical order, which is going to prevent us from double-counting any partitions as we go along.  

Because of this ordering, we can talk about *partial partitions* in a meaningful way.  We define a partial partition to be the truncation of a complete partition.  For example, if $o_1,o_2,o_3,o_4$ is a full partition into four pieces, then $o_1,o_2$ is its (unique) length-two partial partition.  By the construction of the canonical ordering, we can also observe that if $o_i$ and $o_j$ are two ominos which appear together in some plan, then either $o_i$ *always* precedes $o_j$ or $o_j$ *always* precedes $o_i$ in any plan.  This is because if we have some plan where $o_i$ precedes $o_j$, then it must be the case that some cell of $o_i$ occurs before some cell of $o_j$ in the left-to-right, top-to-bottom ordering of the grid, so in every canonical ordering of any partition including both $o_i$ and $o_j$, $o_i$ comes first.

This uniqueness lets us imagine a tree structure on the set of valid partitions.  The root is the empty list, which is the unique length-zero partial partition and is the length-zero partial partition for every partition.  The leaves of the tree are the valid partitions which we are trying to enumerate.  The level-$i$ nodes correspond to length-$i$ partial partitions, and since the canonical ordering of the chunks of a partition induce a well-defined partial order on the ominos, this tree structure is well-defined.

Our next trick to pare down the search space is to build a dictionary of conflicts which tells us which ominos are compatible with which other ominos.  If there are two ominos which overlap or lead to an unfillable hole, it's a huge waste of time to try to explore partial plans which contain both of them, so this dictionary helps us avoid that.  We'll say that two ominos are compatible if they do not overlap and their combination doesn't result in a partial plan which has unfillable holes.

While we're here, we're also going to build a dictionary which, for each entry of the grid, tells us the collection of ominos which have their first one in that entry.  This again lets us narrow down the search space, since we don't want to waste time checking partial plans which have gaps between the $k$th and $k+1$th pieces.


In [ ]:
"""
Given a list of ominos, a list of bad hole sizes, the number
of parts we want to partition into, and a continuity rule,
generates two dictionaries to guide the dynamic program.

It is important that omlist indeed be a list, since we do our
hashing by indexing into that list, so it's order needs
to be fixed.

The first dictionary, firstdict, has a key for each (row,col)
pair to index into the ominos.  The values are the set of ominos
for which first_one(omino)==(row,col).  This will allow the 
algorithm to easily ask for the list of ominos which have their
first one in a particular index.

The second dictionary, conflict, has a key for each omino and
the values are the set of ominos which are 'compatible' with
the key.  Here, compatibility means they do not overlap, and
combining both of them does not create any bad holes.  These
are necessary conditions for the pair of ominos to be able
to be used together in some partitioning.
"""
function build_conflicts(omlist, bad_holes,cont="rc")
    
    
    firstdict = DefaultDict(Set)
    
    for o in 1:length(omlist)
        k = first_one(omlist[o])
        push!(firstdict[k],o)
    end
    
    conflict = DefaultDict(Set)
    for os in subsets(1:length(omlist),Val{2}())
        i = os[1]
        j = os[2]
        _c = true
        for p in zip(omlist[i],omlist[j])
            if _c && p[1]*p[2] ==1
                _c = false
            end
        end
        if _c && check_holes(omlist[i]+omlist[j],bad_holes,cont)
            push!(conflict[i],j)
            push!(conflict[j],i)
        end        
    end
    return firstdict,conflict
end

### How to navigate the tree

We need one last piece before we can be satisfied with the algorithm, and that is to specify how to navigate this tree structure.  Since we read off the ominos from left-to-right, top-to-bottom, we know that any partial partition of length 1 must fill the upper-left corner of the grid.  From there, we can proceed by induction to observe that if we have a length-$k$ partial partition, then any valid partition which includes it must be such that the $k+1$th omino fills the first entry in the grid left empty by the first $k$ pieces.

The next two functions allow us to grab the first zero from a partial plan to match with the first one in a candidate $k+1$th omino.

In [ ]:
"""
Returns the first (lexicographically)
(row,column) index where the entry
of mat is zero.
"""
function first_zero(mat)
    
   for i=1:size(mat,1)
        for j=1:size(mat,2)
            if mat[i,j]==0
                return (i,j)
            end
        end
    end
    return nothing  
end

"""
Returns the first (lexicographically)
(row,column) index where the entry
of mat is one.
"""
function first_one(mat)
    
   for i=1:size(mat,1)
        for j=1:size(mat,2)
            if mat[i,j]==1
                return (i,j)
            end
        end
    end
    return nothing 
end

### The enumerator

Thus, the algorithm proceeeds by recursive depth-first search of the tree, counting all of the leaves it finds.  It begins by starting with the length-one partial partitions, being the set of ominos which fill the upper-left corner of the grid.  For each of these, it recursively finds the length-$k+1$ partial partitions by trying to add each omino which fills the first space left empty by the first $k$ ominos.

In [ ]:
"""
The enumerator. Takes five arguments:
- grid: a pair [m,n] which give the dimension of
        the grid to be partitioned
- om_sizes: a list of integers [a,b,...] which 
            are the allowed sizes of ominos
- num_parts: an integer, the number of parts
- cont: a string "rc" or "qc" to give a continuity rule
- io: a boolean, whether or not to write the list of partitions
      to a file.

returns: an integer, counting the number of valid partitions

usage: to partition the 4x6 grid into 4 rook connected pieces
       of size 4,5,6, with file output, you would call
       enumerator([4,6],[4,5,6],4,"rc",true)

The output format is a text file, where each partition
is on its own line.  The components are numbered 1,2,...num_parts
and the linear order corresponds with the lexicographic order of the
indexing of the grid.  For example, the representation of

1 2 2
1 1 2
3 3 3

is

1,2,2,1,1,2,3,3,3

The file naming convention is [<m>,<n>]_[<p_1>,<p_2>,...,<p_l>]_k_<cont>.txt where 
m and n are the dimensions of the grid, the p_i are the allowed sizes of the chunks, 
k is the number of pieces, and cont is either rc for rook contiguity or qc for queen contiguity.



includes three helper methods defined locally:

verify(a,b) checks if the ominos at indices a,b
conflict with each other as according to the 
conflict dictionary constructed by make_conflicts()

get_next() takes a partial partitioning and gives the
collection of ominos which have their first one where
the partial partition has its first zero and which do not
conflict with any of the ominos in the partial partition

recurs_part() uses get_next() to recursively build
all of the partitions.  If it finds a valid one,
it increments the counter and (if io==true) writes
it to the output file
"""
function enumerator(grid, om_sizes, num_parts, cont="rc", io=false)
    count = 0
    oms = []
    bad_hole_sizes = [ [i for i = 1:minimum(om_sizes)-1] ;[i for i = maximum(om_sizes)+1:2*minimum(om_sizes)-1]        ]
    
    tmp_oms = make_omino_set(1,grid,cont)
    if io
        os = om_sizes
        gr = grid
        np = num_parts
        outfile = open(replace("enumerations/enum_$(gr)_$(os)_$(np)_$(cont).txt"," "=>""),"w")
    end
    
    
    if 1 in om_sizes
        oms = [oms;tmp_oms]
    end
    
    for sz = 2:maximum(om_sizes)
        tmp_oms = biggen(tmp_oms,cont)
        if sz in om_sizes
            oms = [oms;tmp_oms]
        end
        #print("MADE SZ ",sz ,' ',length(tmp_oms),"\n")


    end
    
    
    oms = [ o for o in oms if check_holes(o,bad_hole_sizes,cont)]

    #print("MADE OMINOS\n")
    
    
    
    fd,cf = build_conflicts(oms,bad_hole_sizes,cont)
    #print("MADE DICTIONARIES\n")           
    
    function verify(plan)
        #print(plan,'\n')
        
        for p in subsets(plan,Val{2}())
            if !(p[2] in cf[p[1]])
                return false
            end
        end
        return true
    end
           

    function get_next(fd, curr_inds, omlist) 
       return [d for d in fd[first_zero(sum([omlist[j] for j in curr_inds]))] if verify(curr_inds + d)]     
    end                  
                
    function recurs_part(plan,steps)
        #print("plan:",plan,"  sum: ",sum([oms[j] for j in plan]),'\n')
        if steps == 0
            if sum(sum( [oms[j] for j in plan])) == grid[1]*grid[2] #&& sort!(collect([sum(oms[j]) for j in plan    ] )) == om_sizes
                count +=1
                if io
                    strrep = "$(sum([j.*oms[plan[j]] for j in 1:length(plan)]))\n"
                    strrep = replace(strrep," "=>",")
                    strrep = replace(strrep,"["=>"")
                    strrep = replace(strrep,";"=>"")
                    strrep = replace(strrep,"]"=>"")
                    write(outfile,  strrep)
                end
                return
            end
        elseif sum(sum( [oms[j] for j in plan])) == grid[1]*grid[2]
        return
        end
        for q in fd[first_zero( sum([oms[j] for j in plan     ]))]
            if verify([plan;[q]])
                recurs_part([plan;[q]],steps-1)
            end
        end
    end
    
    #print("NEED TO DO ",length(fd[(1,1)]), " TOP-LEVELS\n")
    for p in fd[(1,1)]
        #print(" ",p," ")
        recurs_part([p],num_parts-1)
    end
    
   print('\n')
    if io
        close(outfile)
    end
   return count 
    
end

### Running the code

We can run the enumerator like this.  I like to use the `@time` macro, which outputs the time it takes for the following function to run.  

This is how you would find the rook contiguous partitions of the $4\times 6$ grid into 4 pieces of size 5, 6, or 7.

In [ ]:
# Run the enumerator. Recall, the signature is
# enumerator([grid size], [omino sizes], number of pieces, contiguity, output)
@time print(enumerator([4,6],[6],4,"rc",false))